<h1 class='font-effect-3d' style='color:firebrick; font-family:Akronim;'>Code Modules, Helpful Functions, Styling, and Links</h1>
[Github Version](https://github.com/OlgaBelitskaya/deep_learning_projects/tree/master/DL_PP5) & [Colaboratory Version](https://colab.research.google.com/drive/1Tt3qZePsf2P6kNNao-hQ58DlG71Abj5a)

In [ ]:
%%html
<style> 
@import url('https://fonts.googleapis.com/css?family=Akronim|Roboto&effect=3d');
body {background-color:#f7e8e8;} 
a,h4 {color:crimson; font-family:Roboto;}
span {color:black; text-shadow:4px 4px 4px #aaa;}
div.output_prompt {color:crimson;} 
div.input_prompt {color:firebrick;} 
div.output_area pre,div.output_subarea {font-size:15px; color:crimson}
div.output_stderr pre {background-color:#f7e8e8;}
</style>

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import numpy as np,pandas as pd,pylab as pl
import h5py,cv2,tensorflow as tf
from tqdm import tqdm
from skimage import color,measure
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential,load_model,Model
from keras.layers import Input,Activation,Dense,LSTM
from keras.layers import Flatten,Dropout,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D,GlobalMaxPooling2D
from keras.layers.advanced_activations import PReLU,LeakyReLU
from keras import __version__
print('keras version:', __version__)
print('tensorflow version:',tf.__version__)
np.set_printoptions(precision=6)
pl.style.use('seaborn-whitegrid')
fw='weights.decor.hdf5'
n=np.random.choice(484,size=6,replace=False)

In [ ]:
def tts(X,y): 
    x_train,x_test,y_train,y_test=\
    train_test_split(X,y,test_size=.2,random_state=1)
    n=int(len(x_test)/2)
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    return x_train,x_valid,x_test,y_train,y_valid,y_test
def history_plot(fit_history):
    pl.figure(figsize=(12,10)); pl.subplot(211)
    keys=list(fit_history.history.keys())[0:4]
    pl.plot(fit_history.history[keys[0]],
            color='crimson',label='train')
    pl.plot(fit_history.history[keys[2]],
            color='firebrick',label='valid')
    pl.xlabel("Epochs"); pl.ylabel("Loss")
    pl.legend(); pl.grid()
    pl.title('Loss Function')     
    pl.subplot(212)
    pl.plot(fit_history.history[keys[1]],
            color='crimson',label='train')
    pl.plot(fit_history.history[keys[3]],
            color='firebrick',label='valid')
    pl.xlabel("Epochs"); pl.ylabel("Accuracy")    
    pl.legend(); pl.grid()
    pl.title('Accuracy'); pl.show()

<h1 class='font-effect-3d' style='color:firebrick; font-family:Akronim;'>Data Loading and Exploration</h1>

In [ ]:
data=pd.read_csv("../input/decor.csv")
data.iloc[n]

In [ ]:
def display_images(img_path,ax):
    img=cv2.imread("../input/decor/"+img_path)
    ax.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))  
fig=pl.figure(figsize=(12,6))
for i in range(6):
    ax=fig.add_subplot(2,3,i+1,xticks=[],yticks=[], 
                       title=data['country'][n[i]]+'; '+\
                       data['decor'][n[i]]+'; '+data['type'][n[i]])
    display_images(data['file'][n[i]],ax)

In [ ]:
f=h5py.File('../input/DecorColorImages.h5','r')
keys=list(f.keys())
[countries,decors,images,types]=\
[np.array(f[keys[i]]) for i in range(4)]
images=images/255
cdecors=to_categorical(np.array(decors-1),7)
sh=[el.shape for el in [images,decors,cdecors,
                        countries,types]]
pd.DataFrame(sh)

In [ ]:
gray_images=np.dot(images[...,:3],[.299,.587,.114])
pl.figure(figsize=(3,3))
n=np.random.choice(484,size=1,replace=False)[0]
pl.imshow(images[n])
pl.title(data['country'][n]+'; '+\
         data['decor'][n]+'; '+data['type'][n])
pl.imshow(gray_images[n],cmap=pl.cm.bone); pl.show()
gray_images=gray_images.reshape(-1,150,150,1)

In [ ]:
# Spliting the data 
# Color Images / Decors 
x_train3,x_valid3,x_test3,\
y_train3,y_valid3,y_test3=tts(images,cdecors)
# Grayscaled Images / Decors 
x_train4,x_valid4,x_test4,\
y_train4,y_valid4,y_test4=tts(gray_images,cdecors)
sh=[el.shape for el in \
[x_train3,y_train3,x_valid3,y_valid3,x_test3,y_test3,
 x_train4,y_train4,x_valid4,y_valid4,x_test4,y_test4]]
pd.DataFrame(sh)

<h1 class='font-effect-3d' style='color:firebrick; font-family:Akronim;'>Neural Networks for One-Label Classification</h1>

In [ ]:
def model(leaky_alpha):
    model=Sequential()
    model.add(Conv2D(32,(5,5),padding='same',
                     input_shape=x_train3.shape[1:]))
    model.add(LeakyReLU(alpha=leaky_alpha))    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(.25))
    model.add(Conv2D(196,(5,5)))
    model.add(LeakyReLU(alpha=leaky_alpha))    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(.25))   
    model.add(GlobalMaxPooling2D())     
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=leaky_alpha))
    model.add(Dropout(.25))   
    model.add(Dense(7))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='nadam',metrics=['accuracy'])    
    return model
model=model(.025)

In [ ]:
lr_reduction=ReduceLROnPlateau(monitor='val_loss',patience=5,
                               verbose=2,factor=.5)
checkpointer=ModelCheckpoint(filepath=fw,verbose=2,save_best_only=True)
estopping=EarlyStopping(monitor='val_loss',patience=15,verbose=2)
history=model.fit(x_train3,y_train3, 
                  epochs=50,batch_size=16,verbose=2,
                  validation_data=(x_valid3,y_valid3),
                  callbacks=[checkpointer,lr_reduction,estopping])

In [ ]:
history_plot(history)
model.load_weights(fw)
model.evaluate(x_test3,y_test3)

In [ ]:
steps,epochs=100,10
generator=ImageDataGenerator(zoom_range=.3,shear_range=.3,
                             rotation_range=30,
                             horizontal_flip=True,vertical_flip=True)
history=model.fit_generator(generator.flow(x_train3,y_train3,batch_size=16),
                            steps_per_epoch=steps,epochs=epochs,verbose=2,
                            validation_data=(x_valid3,y_valid3,), 
                            callbacks=[checkpointer,lr_reduction])

In [ ]:
history_plot(history)
model.load_weights(fw)
model.evaluate(x_test3,y_test3)

In [ ]:
def gray_model(leaky_alpha):
    model=Sequential()
    model.add(Conv2D(32,(5,5),padding='same',
                     input_shape=x_train4.shape[1:]))
    model.add(LeakyReLU(alpha=leaky_alpha))    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(.25))
    model.add(Conv2D(196,(5,5)))
    model.add(LeakyReLU(alpha=leaky_alpha))    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(.25))    
    model.add(GlobalMaxPooling2D())     
    model.add(Dense(1024,activation='tanh'))
    model.add(Dropout(.25))   
    model.add(Dense(7))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model
gray_model=gray_model(.025)

In [ ]:
lr_reduction=ReduceLROnPlateau(monitor='val_loss',patience=5,
                               verbose=2,factor=.7)
checkpointer=ModelCheckpoint(filepath=fw,verbose=2,save_best_only=True)
estopping=EarlyStopping(monitor='val_loss',patience=15,verbose=2)
history=gray_model.fit(x_train4,y_train4, 
                       epochs=50,batch_size=16,verbose=2,
                       validation_data=(x_valid4,y_valid4),
                       callbacks=[checkpointer,lr_reduction,estopping])

In [ ]:
history_plot(history)
gray_model.load_weights(fw)
gray_model.evaluate(x_test4,y_test4)